In [3]:
import numpy as np
from scipy import optimize

In [12]:
# loading input data
data = np.genfromtxt('./data/tt.csv', delimiter=',')
y = data[:,0].reshape(-1,1)
X = data[:, 1:]
m = len(y)

In [28]:
# Some network architecture
input_layer_size  = 784
hidden_layer_size = 28
num_labels = 26

In [81]:
# Creating parameters and a flattened theta
_lambda = 0
theta1 = np.random.rand(hidden_layer_size,785) * 2 * 0.11 - 0.11 #dimension(hidden layer size, input layer size + 1)
theta2 = np.random.rand(hidden_layer_size + 1,num_labels) * 2 * 0.11 - 0.11  #dimension(hiddent layer size + 1, labels)
flat_theta1 = (theta1.T).ravel()
flat_theta2 = (theta2.T).ravel()
nn_params = np.concatenate((flat_theta1, flat_theta2))

#debug stuff
print(f'theta1 shape: {np.shape(theta1)}')
print(f'theta2 shape: {np.shape(theta2)}')
print(theta2[10,10])
print(theta2[20,6])

theta1 shape: (28, 785)
theta2 shape: (29, 26)
0.020924158377765553
-0.02026313823109456


In [90]:
def param_unroll( nn_params, input_layer_size, hidden_layer_size, num_labels ):
    theta1_elems = ( input_layer_size + 1 ) * hidden_layer_size
    theta1_size  = ( input_layer_size + 1, hidden_layer_size  )
    theta2_size  = ( hidden_layer_size + 1, num_labels )

    theta1 = nn_params[:theta1_elems].T.reshape( theta1_size ).T
    theta2 = nn_params[theta1_elems:].T.reshape( theta2_size ).T
    return theta1, theta2

In [91]:
def cost_function(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, _lambda):
    # Reshape nn params back into size
    theta1, theta2 = param_unroll(nn_params, input_layer_size, hidden_layer_size, num_labels)
    print(np.shape(theta1))
    print(np.shape(theta2))
    print(theta2[10,10])
    print(theta2[20,6])

    # forward propogation

In [92]:
J = cost_function(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, _lambda)

(28, 785)
(26, 29)
0.08562544454841618
-0.003983345853868295
